<a href="https://colab.research.google.com/github/ditarnnt/Telecom-Predic-Churn/blob/main/ML_Kaggle_preprocessed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Starting Steps

Training: Run all

Evaluating: Run dari awal sampai feature selection, lalu run 'Evaluation'

In [ ]:
from google.colab import drive
import pandas as pd #pandas
import numpy as np #numpy
import re
from sklearn.preprocessing import LabelEncoder, OneHotEncoder #encoding data kategorikal
from sklearn.preprocessing import StandardScaler #standardization
from sklearn.model_selection import train_test_split #train test split
from sklearn.ensemble import RandomForestClassifier
import pickle
import time

!pip install ipython-autotime
%load_ext autotime

rand_state = 101 #hyperparameter search argument
niter = 125 #hyperparameter search argument
kfold = 3 #hyperparameter search argument

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.55 s (started: 2021-05-11 10:48:52 +00:00)


In [ ]:
drive.mount('/content/drive')
dataset = pd.read_csv('/content/drive/MyDrive/TubesAI_folder/Dataset/df_preprocessed.csv')  
print (dataset)

Mounted at /content/drive
         rev_Mean     mou_Mean  totmrc_Mean  ...  creditcd  kid6_10  churn
0       23.997500   219.250000      22.5000  ...       1.0      0.0    1.0
1       16.990000    10.250000      16.9900  ...       1.0      0.0    1.0
2       55.230000   570.500000      71.9800  ...       1.0      0.0    0.0
3       31.662500    25.500000      29.9900  ...       1.0      0.0    0.0
4      212.515000  1971.500000      84.9900  ...       1.0      1.0    0.0
...           ...          ...          ...  ...       ...      ...    ...
30646   49.362500   187.750000      40.0725  ...       1.0      0.0    1.0
30647   47.490000     0.000000      49.9900  ...       1.0      0.0    1.0
30648   41.840000   351.000000      30.0000  ...       1.0      1.0    0.0
30649   38.340000   319.500000      30.0000  ...       1.0      0.0    1.0
30650   31.923333    63.333333      16.9900  ...       1.0      0.0    0.0

[30651 rows x 89 columns]
time: 29.5 s (started: 2021-05-11 10:48:55 +00:

In [ ]:
def select_features(df,selected_features):
  '''
  Fungsi untuk memilih fitur pada dataset.
  Menggunakan RegEx untuk seleksi fitur yang di-encode secara OneHot Encoding.
  Ketentuan seleksi fitur OneHotEncoding: Fitur harus punya prefix yang sesuai dengan nama fitur.
  Argumen:
  df - dataset yang akan diseleksi fitur-fiturnya
  selected_features - list berisi fitur-fitur yang akan dipilih
  Output:
  df_out - dataset yang berisi fitur-fitur pada selected_features.
  '''
  df_out = pd.DataFrame(index=df.index.tolist())
  cols = []
  for feature in selected_features:
    r = re.compile(feature) # Read Note
    cols = cols + list(filter(r.match,df.columns.tolist()))
  df_out = df[cols]
  return df_out

time: 4.34 ms (started: 2021-05-11 10:49:24 +00:00)


 0.028748    49 months 

 0.024216    76 eqpdays
 
 0.007888    10 change_mou
 
 0.00755     67 hnd_price
 
 0.00579     81 hnd_webcap
 
 0.005635     3 totmrc_Mean
 
 0.005348     2 mou_Mean
 
 0.004054    61 avg3mou
 
 0.003853    62 avg3qty
 
 0.002906    55 adjrev
 
 0.002769    54 totrev
 
 0.002671    52 totcalls
 
 0.002524    57 adjqty
 
 0.002145    11 change_rev

In [ ]:
y = dataset['churn']
X = dataset.drop(['churn'], axis = 1)

time: 11.1 ms (started: 2021-05-11 10:49:24 +00:00)


In [ ]:
print(f'X: {X.shape}')

X: (30651, 88)
time: 1 ms (started: 2021-05-11 10:49:24 +00:00)


In [ ]:
X_train , X_test, y_train, y_test = train_test_split(X, y , test_size = 0.200 , random_state = rand_state)

time: 36.5 ms (started: 2021-05-11 10:49:24 +00:00)


In [ ]:
print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_test: {y_test.shape}')

X_train: (24520, 88)
y_train: (24520,)
X_test: (6131, 88)
y_test: (6131,)
time: 4.65 ms (started: 2021-05-11 10:49:24 +00:00)


In [ ]:
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)

time: 2.61 ms (started: 2021-05-11 10:49:24 +00:00)


# Base Accuracy

In [ ]:
classifier = RandomForestClassifier(random_state = rand_state)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

time: 13.6 s (started: 2021-05-11 10:49:24 +00:00)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print (classification_report(y_test, y_pred))
print (confusion_matrix(y_test, y_pred))
print (accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.67      0.64      3119
           1       0.62      0.56      0.59      3012

    accuracy                           0.62      6131
   macro avg       0.62      0.62      0.62      6131
weighted avg       0.62      0.62      0.62      6131

[[2087 1032]
 [1313 1699]]
0.6175175338443973
time: 31.7 ms (started: 2021-05-11 10:49:38 +00:00)


# Proposed Model: Feature Selection

In [ ]:
'''
selected_features = ['months', 'eqpdays', 'change_mou', 'hnd_price', 'hnd_webcap', 'totmrc_Mean',
                     'moun_Mean', 'avg3mou', 'avg3qty', 'adjrev', 'totrev', 'totcalls', 'adjtqy', 'changerev',
                     'lor', 'asl_flag','ovrrev_Mean','vceovr_Mean','ovrmou_Mean','mou_opkv_Mean','mou_cvce_Mean',
                     'custcare_Mean','mou_peav_Mean','mouiwylisv_Mean','iwylis_vce_Mean','totmou','avg6qty','adjmou',
                     'opk_vce_Mean','avg6mou','peak_vce_Mean','comp_vce_Mean','complete_Mean','cc_mou_Mean',
                     'attempt_Mean','plcd_vce_Mean','ccrndmou_Mean','refurb_news','models','ouowylisv_Mean',
                     'owylis_vce_Mean','uniqsubs','unan_vce_Mean','avgqty','phones','roam_Mean','threeway_Mean',
                     'income''kid0_2', 'churn']
'''

selected_features = ['months', 'eqpdays', 'change_mou', 'hnd_price', 'hnd_webcap', 'totmrc_Mean',
                     'moun_Mean', 'avg3mou', 'avg3qty', 'adjrev','totrev','totcalls','adjtqy','change_rev','churn']

time: 2.98 ms (started: 2021-05-11 10:49:38 +00:00)


In [ ]:
df_out = select_features(dataset, selected_features)
y = df_out['churn']
X = df_out.drop(['churn'], axis = 1)
X_train , X_test, y_train, y_test = train_test_split(X, y , test_size = 0.200 , random_state = rand_state)

time: 19.7 ms (started: 2021-05-11 10:49:38 +00:00)


# Proposed Model: Hyperparameter Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

random_search = {'criterion': ['entropy', 'gini'],
               'max_depth': list(np.linspace(10, 600, 8, dtype = int)) + [None],
               'max_features': ['auto', 'sqrt','log2', None],
               'min_samples_leaf': [4, 6, 8, 12],
               'min_samples_split': [5, 7, 10, 14],
               'n_estimators': list(np.linspace(50, 300, 8, dtype = int))}

clf = RandomForestClassifier()
model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = niter, 
                               cv = kfold, verbose= 5, random_state= rand_state, n_jobs = -1)
model.fit(X_train,y_train) 

Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 26.8min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed: 44.4min
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed: 57.3min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

time: 57min 24s (started: 2021-05-11 10:49:38 +00:00)


In [ ]:
#manualclf = RandomForestClassifier(criterion='gini',
#                                   max_depth=10,
#                                   max_features='sqrt',
#                                   min_samples_leaf=12,
#                                   min_samples_split=7,
#                                   n_estimators=300)
#manualclf.fit(X_train,y_train) 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=12, min_samples_split=7,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

time: 12.6 s (started: 2021-05-11 11:47:03 +00:00)


In [ ]:
predictionforest = model.best_estimator_.predict(X_test)
print(confusion_matrix(y_test,predictionforest))
print(classification_report(y_test,predictionforest))
acc = accuracy_score(y_test,y_pred)
acc1 = accuracy_score(y_test,predictionforest)

#predictionforest = manualclf.predict(X_test)
#print(confusion_matrix(y_test,predictionforest))
#print(classification_report(y_test,predictionforest))
#acc = accuracy_score(y_test,y_pred)
#acc1 = accuracy_score(y_test,predictionforest)

[[2042 1077]
 [1250 1762]]
              precision    recall  f1-score   support

         0.0       0.62      0.65      0.64      3119
         1.0       0.62      0.58      0.60      3012

    accuracy                           0.62      6131
   macro avg       0.62      0.62      0.62      6131
weighted avg       0.62      0.62      0.62      6131

time: 158 ms (started: 2021-05-11 11:49:48 +00:00)


In [ ]:
print("Base accuracy:",acc*100,"%")
print("Optimized accuracy:",acc1*100,"%")
print("Increase in accuracy:",(acc1-acc)*100,"%")

Base accuracy: 61.75175338443973 %
Optimized accuracy: 62.04534333713912 %
Increase in accuracy: 0.29358995269939125 %
time: 7.66 ms (started: 2021-05-11 11:49:53 +00:00)


# Save Base & Proposed Model

In [ ]:
#ngesave model

runtime = time.strftime("%d-%m-%Y %H:%M:%S")

base_filename = "/content/drive/MyDrive/TubesAI_folder/Model_df/df_base_" + runtime + ".pkl"
with open(base_filename, 'wb') as file:
  pickle.dump(classifier, file)

pkl_filename = "/content/drive/MyDrive/TubesAI_folder/Model_df/df_model_" + runtime + ".pkl"
with open(pkl_filename, 'wb') as file:
  pickle.dump(manualclf, file)

with open("/content/drive/MyDrive/TubesAI_folder/Model_df/log_" + runtime + ".txt",'a') as file:
  L = ["base accuracy = "+str(acc*100)+"%\n",
  "opt accuracy = "+str(acc1*100)+"%\n",
  "Increase in accuracy:"+str((acc1-acc)*100)+"%\n",
  str(model.best_estimator_.get_params())]
  file.writelines(L)  

time: 287 ms (started: 2021-05-11 11:50:00 +00:00)


In [ ]:
model.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 10,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 8,
 'min_samples_split': 14,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 157,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

time: 6.99 ms (started: 2021-05-11 11:50:09 +00:00)


# Evaluation

In [ ]:
infile = open('/content/drive/MyDrive/TubesAI_folder/Model_df/df_model_'+runtime+'.pkl','rb')
evalmodel = pickle.load(infile)
infile.close()

evalmodel.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 10,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 12,
 'min_samples_split': 7,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 300,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

time: 45.9 ms (started: 2021-05-11 11:50:14 +00:00)


In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score

y_eval = evalmodel.predict(X_test)

acceval = accuracy_score(y_test,y_eval)*100
preciseval = precision_score(y_test,y_eval)*100
recalleval = recall_score(y_test,y_eval)*100
f1eval = f1_score(y_test,y_eval)*100

print("EVALUATION OF PROPOSED MODEL:\n")

print("Summary:")
print(classification_report(y_test,y_eval))

print("Confusion matrix:")
print(str(confusion_matrix(y_test,y_eval))+"\n")

print("Accuracy:")
print(str(acceval)+"%\n")

print("Precision:")
print(str(preciseval)+"%\n")

print("Recall:")
print(str(recalleval)+"%\n")

print("F1 Score:")
print(str(f1eval)+"%\n")

EVALUATION OF PROPOSED MODEL:

Summary:
              precision    recall  f1-score   support

         0.0       0.62      0.65      0.64      3119
         1.0       0.62      0.59      0.60      3012

    accuracy                           0.62      6131
   macro avg       0.62      0.62      0.62      6131
weighted avg       0.62      0.62      0.62      6131

Confusion matrix:
[[2030 1089]
 [1240 1772]]

Accuracy:
62.01272223128365%

Precision:
61.93638587906326%

Recall:
58.831341301460824%

F1 Score:
60.3439468755321%

time: 279 ms (started: 2021-05-11 11:50:25 +00:00)


In [ ]:
accpred = accuracy_score(y_test,y_pred)*100
precispred = precision_score(y_test,y_pred)*100
recallpred = recall_score(y_test,y_pred)*100
f1pred = f1_score(y_test,y_pred)*100

print("EVALUATION OF BASE MODEL:\n")

print("Summary:")
print(classification_report(y_test,y_pred))

print("Confusion matrix:")
print(str(confusion_matrix(y_test,y_pred))+"\n")

print("Accuracy:")
print(str(accpred)+"%\n")

print("Precision:")
print(str(precispred)+"%\n")

print("Recall:")
print(str(recallpred)+"%\n")

print("F1 Score:")
print(str(f1pred)+"%\n")

EVALUATION OF BASE MODEL:

Summary:
              precision    recall  f1-score   support

         0.0       0.61      0.67      0.64      3119
         1.0       0.62      0.56      0.59      3012

    accuracy                           0.62      6131
   macro avg       0.62      0.62      0.62      6131
weighted avg       0.62      0.62      0.62      6131

Confusion matrix:
[[2087 1032]
 [1313 1699]]

Accuracy:
61.75175338443973%

Precision:
62.21164408641523%

Recall:
56.407702523240374%

F1 Score:
59.167682395960306%

time: 46.1 ms (started: 2021-05-11 11:50:28 +00:00)
